# Gensim, JUMAN++, KNPで文書類似度算出

[DeepAgeさんの「Doc2Vecの仕組みとGensimを使った文書類似度算出チュートリアル」](https://deepage.net/machine_learning/2017/01/08/doc2vec.html#最も似ている記事を取得する)
が大変参考になりました。ありがとうございました！

注意：
- JUMAN++, KNPは[Ansibleタスク](../roles)でインストールできるようにしてあります
- KNPのバージョンが4.17に上がりました
- ファイルを開くとき、文字コードをutf-8に指定する必要があります

## 参考：

- [DeepAge Doc2Vecの仕組みとGensimを使った文書類似度算出チュートリアル](https://deepage.net/machine_learning/2017/01/08/doc2vec.html#最も似ている記事を取得する)
- [黒橋・河原研究室　自然言語処理のためのリソース](http://nlp.ist.i.kyoto-u.ac.jp/index.php?NLP%E3%83%AA%E3%82%BD%E3%83%BC%E3%82%B9)
- [hassaku's blog 学習済みword2vecモデルを調べてみた](http://blog.hassaku-labs.com/post/pretrained-word2vec/)
- [白ヤギコーポレーション word2vecの学習済み日本語モデルを公開します](http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/)

## コーパス

livedoorのニュースコーパスをダウンロード

In [ ]:
!export LDCC=ldcc-20140209 &&\
mkdir -p /workspace/notebooks/corpora/ldcc &&\
cd /workspace/notebooks/corpora/ldcc &&\
wget http://www.rondhuit.com/download/$LDCC.tar.gz &&\
tar xvfz $LDCC.tar.gz &&\
rm $LDCC.tar.gz

## 助走

必要ライブラリをimportし、関数をいくつか定義

In [2]:
import sys
from os import listdir, path
from pyknp import Jumanpp
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

記事ファイルをダウンロードしたディレクトリから取得する関数

In [3]:
def corpus_files():
    dirs = [path.join('corpora/ldcc/text', x)
            for x in listdir('corpora/ldcc/text') if not x.endswith('.txt')]
    docs = [path.join(x, y)
            for x in dirs for y in listdir(x) if not x.startswith('LICENSE')]
    return docs

記事コンテンツをパスから取得する関数

In [4]:
def read_document(path):
    with open(path, 'r', encoding="utf-8") as f:
        return f.read()

JUMAN++を使って記事を単語リストに変換する関数

In [5]:
def split_into_words(text):
    result = Jumanpp().analysis(text)
    return [mrph.midasi for mrph in result.mrph_list()]

## 形態素解析

記事コンテンツを単語に分割して、Doc2Vecの入力に使うLabeledSentenceに変換する関数

In [6]:
def doc_to_sentence(doc, name):
    words = split_into_words(doc)
    return LabeledSentence(words=words, tags=[name])

記事のパスリストから、記事コンテンツに変換し、単語分割して、センテンスのジェネレーターを返す関数

In [7]:
def corpus_to_sentences(corpus):
    docs   = [read_document(x) for x in corpus]
    for idx, (doc, name) in enumerate(zip(docs, corpus)):
        sys.stdout.write('\r前処理中 {}/{}'.format(idx, len(corpus)))
        yield doc_to_sentence(doc, name)

Doc2Vecパラメータを渡して、学習

In [8]:
corpus = corpus_files()
sentences = corpus_to_sentences(corpus)

## 学習

dmに1を設定するとdmpvで学習されることになる。1以外であれば、DBoWで学習される。

In [9]:
model = Doc2Vec(sentences, dm=0, size=300, window=15, alpha=.025,
        min_alpha=.025, min_count=1, sample=1e-6)

前処理中 7375/7376

In [ ]:
print('\n訓練開始')
for epoch in range(20):
    print('Epoch: {}'.format(epoch + 1))
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= (0.025 - 0.0001) / 19
    model.min_alpha = model.alpha

print('モデル保存\n')
!mkdir -p models
model.save('models/doc2vec.model')

モデルの保存と読み込み

In [11]:
# モデル読み込み
model = Doc2Vec.load('models/doc2vec.model')

## 類似記事の取得


最も似ている記事を取得する

In [12]:
model.docvecs.most_similar('corpora/ldcc/text/livedoor-homme/livedoor-homme-5625149.txt', topn=1)

[('corpora/ldcc/text/movie-enter/movie-enter-6165056.txt',
  0.24260929226875305)]

## 類似度の算出

In [13]:
model.docvecs.similarity('corpora/ldcc/text/livedoor-homme/livedoor-homme-4700669.txt', 
                         'corpora/ldcc/text/movie-enter/movie-enter-5947726.txt')

-0.11023943942407691